# AIFFEL LMS Exploration 7

    7-9. 프로젝트 - Movielens 영화 추천 실습

 * MF 모델 학습 방법을 토대로 영화 추천 시스템 제작
 * 데이터 셋은 Movielens 사용(추천 시스템의 MNIST)
 
     * 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있음. 
     * MovieLens 1M Dataset 사용을 권장합니다.
     * 별점 데이터는 대표적인 explicit(명시적) 데이터입니다. 
     * 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
     * 별점을 시청횟수로 해석해서 생각하겠습니다.
     * 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
 

# 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 

완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
# 라이브러리 호출

import os
import pandas as pd

In [3]:
# 데이터 준비

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# # rating 컬럼의 이름을 count로 바꿉니다.
# ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2) 분석해 봅시다.

 * ratings에 있는 유니크한 영화 개수
 * ratings에 있는 유니크한 사용자 수
 * 가장 인기있는 영화 30개(인기순)

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자의 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기있는 영화 30개(인기순)

movie_count = ratings.groupby('movie_id')['user_id'].count()

# ascending=False(내림차순)
favorite_30 = movie_count.sort_values(ascending=False).head(30)

In [10]:
# Movie_id를 이름으로 변환

merge_movie = pd.merge(favorite_30, movies, on='movie_id')
favorite_30 = merge_movie[['title', 'user_id']]
favorite_30.rename(columns={'user_id': 'movie_count'}, inplace=True)
favorite_30

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,title,movie_count
0,American Beauty (1999),3211
1,Star Wars: Episode IV - A New Hope (1977),2910
2,Star Wars: Episode V - The Empire Strikes Back...,2885
3,Star Wars: Episode VI - Return of the Jedi (1983),2716
4,Saving Private Ryan (1998),2561
5,Terminator 2: Judgment Day (1991),2509
6,"Silence of the Lambs, The (1991)",2498
7,Raiders of the Lost Ark (1981),2473
8,Back to the Future (1985),2460
9,"Matrix, The (1999)",2434


### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [11]:
# ratings에 영화 제목 열 추가
ratings = pd.merge(ratings, movies, on='movie_id')

# 불필요 열 제거
using_cols = ['user_id', 'rating', 'title']
ratings = ratings[using_cols]

ratings.head()

,user_id,rating,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,2,5,One Flew Over the Cuckoo's Nest (1975)
2,12,4,One Flew Over the Cuckoo's Nest (1975)
3,15,4,One Flew Over the Cuckoo's Nest (1975)
4,17,5,One Flew Over the Cuckoo's Nest (1975)


In [12]:
# 내가 좋아하는 영화 5가지 추가
my_favorite = ['Terminator 2: Judgment Day (1991)', 'Toy Story (1995)', 'Matrix, The (1999)', 'Sixth Sense, The (1999)', 'Men in Black (1997)']

# 'sangun'이라는 user_id가 위 영화에 rating 점수를 각 5점씩 주었다고 가정하겠습니다.
my_ratings = pd.DataFrame({'user_id': ['sangun']*5, 'title' : my_favorite, 'rating' : [5]*5})

if not ratings.isin({'user_id':['sangun']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_ratings)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,rating,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,sangun,5,Terminator 2: Judgment Day (1991)
1,sangun,5,Toy Story (1995)
2,sangun,5,"Matrix, The (1999)"
3,sangun,5,"Sixth Sense, The (1999)"
4,sangun,5,Men in Black (1997)


In [14]:
# 해당 내용을 가지고 CSR Matrix를 만드려면
# data의 요소가 정수형이어야 함
# 따라서 indexing 작업이 필요

# indexing 작업
# user_id와 title을 고유한 값으로 따로 분리 한 후,
# 분리한 Series(배열인가??)의 index값과 value를 전환하여
# values를 정수 값으로, index를 name으로 변경

# user_id와 title을 고유한 값으로 따로 분리
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

In [15]:
# index와 name을 전환(index를 name으로, value를 숫자(이전 index)으로 변환)
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()

# 모든 row가 정상적으로 인덱싱되었다면
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    
    # data['user_id']을 인덱싱된 Series로 교체
    ratings['user_id'] = temp_user_data   
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing Fail!!
title column indexing OK!!


,user_id,rating,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,92
1,6039,5,40
2,6039,5,124
3,6039,5,38


# 4) CSR matrix를 직접 만들어 봅시다.

In [18]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie_title = ratings['title'].nunique()

csr_data_movie = csr_matrix((ratings.rating, (ratings.user_id, ratings.title)), shape= (num_user, num_movie_title))
csr_data_movie

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_movie_transpose = csr_data_movie.T
csr_data_movie_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_movie_transpose)

In [23]:
als_model??


# 6) 내가 선호하는 5가지 영화 중 하나와 
# 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [29]:
# als_model모델로 부터 나(사용자)의 벡터와
# 영화 Toy_story(내가 선호하는 영화) 벡터를 추출
# 영화 One Little Indian (1973)(그외 영화) 벡터를 추출
sangun, Toy_story, One_Little_Indian = user_to_idx['sangun'], movie_to_idx['Toy Story (1995)'], movie_to_idx['One Little Indian (1973)']
sangun_vector, Toy_story_vector, One_Little_Indian_vector = als_model.user_factors[sangun], als_model.item_factors[Toy_story], als_model.item_factors[One_Little_Indian]

In [30]:
# 나의 벡터
sangun_vector

array([-0.5603625 ,  0.4505748 ,  0.5669001 , -0.23003362, -0.4039458 ,
        0.25943407,  0.39498255,  0.56650174, -1.1576502 ,  0.02584512,
       -0.8501873 ,  0.36202702, -0.28842062,  0.55230224, -0.5712281 ,
        0.7837369 ,  0.39638746,  0.49892867, -0.8129371 , -0.00946595,
       -0.23912163,  0.02114581, -0.6924136 ,  0.37901032,  0.7843585 ,
       -0.46787757,  0.5032639 ,  0.9295692 ,  0.7023011 , -0.25353137,
        0.3353605 ,  0.54044974, -0.3794056 ,  0.14654009,  0.03090282,
        0.8854265 , -0.51292354,  0.4609752 ,  0.4359431 , -0.50351065,
       -0.3979103 ,  0.08474804,  0.0824995 ,  0.18044755,  0.9975338 ,
        0.24860363,  0.8919879 , -0.02578691, -0.80409443, -0.5666247 ,
       -0.3778695 ,  0.32773507,  0.3069794 , -0.40734628, -0.81897765,
        0.25485742,  1.2103451 , -0.3242189 , -0.60826606, -0.09306651,
       -0.28754961,  0.44828126, -0.52722204, -0.09816736, -1.0993963 ,
       -0.00833315, -1.1595945 , -0.05867288,  0.32855895,  1.24

In [31]:
# Toy_story 가수 벡터
Toy_story_vector

array([ 7.13703269e-03,  2.63742022e-02,  4.52953428e-02, -8.17725714e-03,
        1.67649090e-02,  3.96982096e-02, -9.01611988e-03,  1.16119031e-02,
        1.74497603e-03,  2.18468811e-02, -2.12168992e-02,  5.68462582e-03,
       -4.26446125e-02,  1.18340785e-02,  1.43269654e-02,  2.92421859e-02,
        2.25068983e-02,  3.98178436e-02, -1.30464863e-02,  3.12403291e-02,
       -7.74481893e-03, -9.09788650e-05,  1.54506089e-03,  1.18601648e-02,
       -1.20176459e-02,  3.19792819e-03, -7.87341222e-03,  2.14771181e-02,
        1.94216706e-02, -4.89063794e-03,  2.19762754e-02,  1.22379400e-02,
        1.49826054e-02,  1.55848414e-02, -2.54235733e-02,  4.82536331e-02,
       -1.42904306e-02, -2.25529005e-03, -1.51273506e-02,  2.01046150e-02,
       -1.61800645e-02, -5.70725696e-03, -2.15578340e-02,  1.82824824e-02,
        6.31600711e-03,  4.12444510e-02,  6.09971164e-03, -9.04338807e-03,
       -2.75884084e-02, -6.92073163e-03,  1.55285038e-02, -1.74275450e-02,
        2.32735742e-02, -

In [32]:
# One_Little_Indian 가수 벡터
One_Little_Indian_vector

array([-4.6763028e-04,  2.8291857e-04,  5.2626734e-04,  1.6085879e-03,
       -6.9138128e-04, -9.5952731e-05,  2.3168002e-04, -5.2222254e-04,
       -9.8100409e-04,  1.4966616e-03,  5.0299644e-04,  6.2647095e-04,
        5.4632616e-04,  8.9736801e-05,  1.2930005e-04, -4.0758331e-04,
        3.7638287e-04,  9.5688790e-04,  1.2688237e-04,  1.7500592e-03,
       -4.8367507e-04, -2.9834881e-04, -2.3389686e-05, -6.9386180e-05,
        1.4379718e-03, -1.8887175e-04,  2.0810748e-04,  1.6544199e-03,
       -1.2747499e-03,  6.8973511e-04,  4.3660228e-04, -1.2534907e-03,
        8.8303391e-04,  1.0226202e-03, -9.0659206e-04, -7.4494301e-06,
        8.7843469e-04,  6.3425483e-05,  1.5699181e-03, -9.6011453e-04,
        4.6348851e-04,  9.6714939e-05,  5.5746210e-04,  1.0443524e-03,
        4.6594400e-04,  5.9169182e-04,  7.7074411e-04, -7.0355285e-04,
        1.6411262e-03,  5.8945554e-04,  1.1384265e-03, -5.4607133e-04,
        1.0711213e-03,  5.0631189e-04,  1.1671166e-03,  8.9781760e-04,
      

In [33]:
# 내가 선호하는 영화(Toy_story)에 대한 선호도 파악

# 나(사용자)의 벡터와 Toy_story 벡터를 내적하는 코드
# 여기서 내가 Toy_story를 선호한다는 결과(=1)를 목표로 학습해야 함
np.dot(sangun_vector, Toy_story_vector)

0.47708407

In [34]:
# 내가 선호하는 5가지 영화 외의 다른 영화(One_Little_Indian)에 대한 선호도 파악

np.dot(sangun_vector, One_Little_Indian_vector)

0.0024401315

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [36]:
# 내가 좋아하는 영화 : Toy_story

favorite_movie = 'Toy Story (1995)'
movies_id = movie_to_idx[favorite_movie]

# N : 추천받을 영화 갯수
similar_movie = als_model.similar_items(movies_id, N=15)
similar_movie

# similart_items 메서드 : 비슷한 아이템을 찾아서
# 아티스트의 id와 유사도를 Tuple로 반환

[(40, 0.19158348),
 (50, 0.15377776),
 (33, 0.11105538),
 (4, 0.10699938),
 (322, 0.098455995),
 (110, 0.096618906),
 (330, 0.08732538),
 (20, 0.08041665),
 (10, 0.077070616),
 (255, 0.06892995),
 (160, 0.066561304),
 (126, 0.062027484),
 (545, 0.061404835),
 (16, 0.06125774),
 (32, 0.060769737)]

In [38]:
# # 데이터 확인용
# type(similar_movie)

list

In [37]:
# 추천 받은 영화 ID를 다시 이름으로 매핑

# movie_to_idx의 key(영화이름)와 value(영화 idx)를 다시 뒤집어 
# key = 영화 idx, value=영화이름 으로 저장
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]
# similar_movie의 각 요소(튜플)의 첫 번째 요소(영화idx)를 i[0]으로 불러오고
# 그 영화 idx에 매핑되는 영화 제목을 idx_to_movie로 불러와 리스트의 요소로 저장

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)",
 'Forrest Gump (1994)',
 'Shakespeare in Love (1998)',
 'Santa Clause, The (1994)',
 'Tarzan (1999)',
 'Hercules (1997)']

In [43]:
# 데이터 확인용
# idx_to_movie

{0: "One Flew Over the Cuckoo's Nest (1975)",
 1: 'James and the Giant Peach (1996)',
 2: 'My Fair Lady (1964)',
 3: 'Erin Brockovich (2000)',
 4: "Bug's Life, A (1998)",
 5: 'Princess Bride, The (1987)',
 6: 'Ben-Hur (1959)',
 7: 'Christmas Story, A (1983)',
 8: 'Snow White and the Seven Dwarfs (1937)',
 9: 'Wizard of Oz, The (1939)',
 10: 'Beauty and the Beast (1991)',
 11: 'Gigi (1958)',
 12: 'Miracle on 34th Street (1947)',
 13: "Ferris Bueller's Day Off (1986)",
 14: 'Sound of Music, The (1965)',
 15: 'Airplane! (1980)',
 16: 'Tarzan (1999)',
 17: 'Bambi (1942)',
 18: 'Awakenings (1990)',
 19: 'Big (1988)',
 20: 'Pleasantville (1998)',
 21: 'Wallace & Gromit: The Best of Aardman Animation (1996)',
 22: 'Back to the Future (1985)',
 23: "Schindler's List (1993)",
 24: 'Meet Joe Black (1998)',
 25: 'Pocahontas (1995)',
 26: 'E.T. the Extra-Terrestrial (1982)',
 27: 'Titanic (1997)',
 28: 'Ponette (1996)',
 29: 'Close Shave, A (1995)',
 30: 'Antz (1998)',
 31: 'Girl, Interrupted (199

In [44]:
# # 데이터 확인용
# idx_to_movie[0]

"One Flew Over the Cuckoo's Nest (1975)"

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

AlternatingLeastSquares 클래스에 구현되어 있는 
recommend 메서드 이용

In [45]:
# 'sangun'의 idx값을 가져옴
user = user_to_idx['sangun']


# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data_movie, N=20, filter_already_liked_items=True)
movie_recommended

[(107, 0.6527393),
 (62, 0.44390023),
 (141, 0.43982577),
 (50, 0.36533713),
 (121, 0.35032588),
 (670, 0.3223219),
 (200, 0.3174786),
 (60, 0.30255914),
 (110, 0.27575082),
 (75, 0.27196938),
 (22, 0.25415683),
 (160, 0.23862839),
 (145, 0.23575914),
 (64, 0.22905965),
 (375, 0.221074),
 (82, 0.21701594),
 (317, 0.21480972),
 (44, 0.21026692),
 (4, 0.20858629),
 (150, 0.20734999)]

In [46]:
# movie idx를 영화 제목으로 다시 매핑
[idx_to_movie[i[0]] for i in movie_recommended]

['Jurassic Park (1993)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Toy Story 2 (1999)',
 'Silence of the Lambs, The (1991)',
 'Galaxy Quest (1999)',
 'Terminator, The (1984)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Groundhog Day (1993)',
 'Hunt for Red October, The (1990)',
 'Back to the Future (1985)',
 'Forrest Gump (1994)',
 'Fifth Element, The (1997)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Face/Off (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Twelve Monkeys (1995)',
 'Star Wars: Episode IV - A New Hope (1977)',
 "Bug's Life, A (1998)",
 'Independence Day (ID4) (1996)']

# 결과 및 느낀점

 * 결과
     - CSR Matrix가 정상적으로 생성되고
     - MF 모델을 통하여 훈련 및 추천을 통해 선호하는 영화와 유사 영화 및
       사용자(나)에 대한 추천 영화가 잘 나타나서 잘 수행된 것 같습니다.
       

 * 느낀점
     - 데이터가 90년대 외화를 기반이어서 알고 있는 영화가 많지 않은점이 아쉬웠습니다.
       만일, 비교적 최신 영화와 국내 영화에 대한 데이터를 얻을 수 있다면, 
       보다 알고리즘의 성능이 직접적으로 와닿을 것 같습니다.
     - 영화 index값과 영와 제목으로 서로 매핑하여 전환하는 과정이 처음에 어려웠는데,
       퍼실님 설명을 통해 어렵지만, 간결하고 유용한 문법을 잘 숙지하게 되었습니다.